In [54]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from urllib.parse import urlparse


In [63]:
def getData(urls):

    response = requests.get(urls)
    
    if response.status_code == 200:

        soup = BeautifulSoup(response.text,"lxml")

        list_of_books = soup.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
        data = []

        for book in list_of_books:

            imgUrl = book.find("div",class_="image_container")
            img = imgUrl.find("img")['src']

            completeUrl = f"https://books.toscrape.com/{img}"

            title = book.find("h3")
            price = book.find("p",class_="price_color")
            clean_price = price.text.replace("Â", "").strip()
            availabilty = book.find("p",class_="instock availability")
            
            row = [title.text.strip(),clean_price,availabilty.text.strip(),completeUrl]

            data.append(row)
    nextpage = soup.find("li",class_="next")

    if nextpage:

        pageUrl = nextpage.find("a")['href']
        parsed_url = urlparse(pageUrl)
        page = parsed_url.path.split("/")[-1]
        nextpageUrl = "https://books.toscrape.com/catalogue/"+page
        data += getData(nextpageUrl)

    return data
 

url = "https://books.toscrape.com/"

book_df = getData(url)
columns = ['Title' , 'Price' , 'Availability' , 'Book Image' ]

final_book_df = pd.DataFrame(book_df,columns=columns)

final_book_df.to_excel("books_data.xlsx",index=False)


[None, None, None]